In [220]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.neural_network import MLPClassifier
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [221]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_attrition.h5')


In [222]:
application_df = application_df.drop(['EIN', 'NAME', 'APPLICATION_TYPE', 'SPECIAL_CONSIDERATIONS'],1)
application_df.head(10)

C:\Users\antho\anaconda3\envs\PythonData\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,Independent,C1000,ProductDev,Association,1,0,5000,1
1,Independent,C2000,Preservation,Co-operative,1,1-9999,108590,1
2,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0
3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,6692,1
4,Independent,C1000,Heathcare,Trust,1,100000-499999,142590,1
5,Independent,C1200,Preservation,Trust,1,0,5000,1
6,Independent,C1000,Preservation,Trust,1,100000-499999,31452,1
7,Independent,C2000,Preservation,Trust,1,10M-50M,7508025,1
8,Independent,C1000,ProductDev,Trust,1,1-9999,94389,1
9,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0


In [223]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df.CLASSIFICATION.value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [224]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_count[classification_count < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [225]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT']

In [226]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\antho\anaconda3\envs\PythonData\envs\PythonData\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [227]:
# Merge one-hot encoded features and drop the originals
application_df= application_df.merge(encode_df,left_index=True,right_index=True)
application_df = application_df.drop(columns = application_cat)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [228]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"]
X = application_df.drop(columns="IS_SUCCESSFUL")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 50)


In [229]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [230]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 20
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 60
hidden_nodes_layer4 = 80
hidden_nodes_layer5 = 100



nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third Hidden Layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Fourth Hidden Layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

#5th Hidden Layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_82 (Dense)            (None, 20)                660       
                                                                 
 dense_83 (Dense)            (None, 40)                840       
                                                                 
 dense_84 (Dense)            (None, 60)                2460      
                                                                 
 dense_85 (Dense)            (None, 80)                4880      
                                                                 
 dense_86 (Dense)            (None, 100)               8100      
                                                                 
 dense_87 (Dense)            (None, 1)                 101       
                                                                 
Total params: 17,041
Trainable params: 17,041
Non-tra

In [231]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [232]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)


Epoch 1/100
804/804 [==============================] - 1s 936us/step - loss: 0.6934 - accuracy: 0.5225
Epoch 2/100
804/804 [==============================] - 1s 914us/step - loss: 0.6926 - accuracy: 0.5272
Epoch 3/100
804/804 [==============================] - 1s 928us/step - loss: 0.6921 - accuracy: 0.5246
Epoch 4/100
804/804 [==============================] - 1s 925us/step - loss: 0.6922 - accuracy: 0.5243
Epoch 5/100
804/804 [==============================] - 1s 933us/step - loss: 0.6922 - accuracy: 0.5265
Epoch 6/100
804/804 [==============================] - 1s 895us/step - loss: 0.6925 - accuracy: 0.5266
Epoch 7/100
804/804 [==============================] - 1s 909us/step - loss: 0.6922 - accuracy: 0.5268
Epoch 8/100
804/804 [==============================] - 1s 944us/step - loss: 0.6925 - accuracy: 0.5219
Epoch 9/100
804/804 [==============================] - 1s 887us/step - loss: 0.6924 - accuracy: 0.5246
Epoch 10/100
804/804 [==============================] - 1s 887us/step - l

804/804 [==============================] - 1s 1ms/step - loss: 0.6924 - accuracy: 0.5254
Epoch 81/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6918 - accuracy: 0.5295
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6920 - accuracy: 0.5291
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6918 - accuracy: 0.5313
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6921 - accuracy: 0.5277
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6918 - accuracy: 0.5292
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6917 - accuracy: 0.5299
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6919 - accuracy: 0.5293
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6921 - accuracy: 0.5279
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6921 - accuracy: 

In [233]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6911 - accuracy: 0.5349 - 372ms/epoch - 1ms/step
Loss: 0.6911366581916809, Accuracy: 0.5349271297454834


In [234]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization_attempt1.h5")